Again, before start, let's import a few Python modules.

This time we will focus on a new module of `kongming.memory`.

In [1]:
from kongming import api, hv, memory 
from kongming.hv import helpers

GOMAXPROCS set to 128 for this invocation.


# Interacts with memory

Once we have a good understanding of how hypervectors and their interactions, the next logical step is to store and retrieve them from a memory. 

Memory is a component that provides efficient storage and near-neighbor search capability for hypervectors. This is analogous to the vector database as discussed in industry. 